# 分點明細查詢

In [313]:
import bs4, lxml
from bs4 import BeautifulSoup
from lxml import etree
import requests

from selenium import webdriver
from selenium.webdriver.support.select import Select
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By


import time, os, pickle, re, glob
import pandas as pd
import numpy as np

import datetime
from datetime import timedelta

#! pip install chinesecalendar
import chinese_calendar
from chinese_calendar import is_workday

import warnings
warnings.filterwarnings('ignore')

## 號碼對應的分點

In [305]:
%%time
chromedriver_autoinstaller.install()
chrome_options = webdriver.ChromeOptions()
browser = webdriver.Chrome(options=chrome_options)

url = 'https://stockchannelnew.sinotrade.com.tw/z/zg/zgb/zgb0.djhtm?a=9200&b=9268&c=B&e=2022-8-2&f=2022-8-2' #隨便抓一天
browser.get(url)
html = browser.page_source
soup = BeautifulSoup(html, "html.parser")

# 號碼對應分點    ex: 9200 -> 凱基、 9268 -> 凱基-台北
dots1 = soup.select('#oScrollHead > td > select:nth-child(2) > option')
number1 = [str(dots1[i]).split('>')[0].split('\"')[-2] for i in range(len(dots1))]
name1 = [str(dots1[i]).split('>')[1].split('<')[0] for i in range(len(dots1))]
#table1 = pd.DataFrame({'號碼1':number1, '分點1':name1})
#print(table1)

dots2 = soup.select('#oScrollHead > td > select:nth-child(3) > option')
number2 = [str(dots2[i]).split('>')[0].split('\"')[-2] for i in range(len(dots2))]
name2 = [str(dots2[i]).split('>')[1].split('<')[0] for i in range(len(dots2))]
#table2 = pd.DataFrame({'號碼2':number2, '分點2':name2})
#print(table2)


Wall time: 1min 1s


## 查詢函式

In [368]:
def query(a='9200', b='9268', e=datetime.datetime.now().date()):
    '''
    a: 分點1              #default:凱基
    b: 分點2              #default:凱基-台北
    e: 日期 ex:2022-7-6   #default:今天
    
    return: 買超張數、賣超張數、買超金額、賣超金額
    '''
    try:
        files = []
        for c in ["E", "B"]:  # c=E:張數、 c=B:金額
            url = f'https://stockchannelnew.sinotrade.com.tw/z/zg/zgb/zgb0.djhtm?a={a}&b={b}&c={c}&e={e}&f={str(e)}'
            resp = requests.request(method='GET', url=url)
            if resp.ok :
                data = pd.read_html(resp.text, header=1)

                if data[1]['券商名稱'][0]== '無此券商分點交易資料':
                    #print(f'{date} 無此券商分點交易資料')
                    return None
                else:
                    for i in range(np.shape(data[1])[0]):
                        # data1:買超、data2:賣超
                        buyname = data[1]['券商名稱'][i]
                        sellname = data[2]['券商名稱'][i]
                        if buyname[0]== '<':
                            data[1]['券商名稱'][i] = buyname.split('(')[1].split(')')[0].replace('\'', '').replace(',', '') 
                        if sellname[0]== '<':
                            data[2]['券商名稱'][i] = sellname.split('(')[1].split(')')[0].replace('\'', '').replace(',', '') 
                    files.append(data[1])
                    files.append(data[2])

            else :
                print(f'連線失敗 : 【{resp.status_code}】 : {resp.reason}')

        return files 
    
    except Exception as e:
        print('【系統訊息】{}'.format(e))
        return None
    
# 函式匯出的4個工作表名字
sheet = ['買超張數', '賣超張數', '買超金額', '賣超金額']

### 自訂分點、自動建立資料夾 & 要跑的日期

In [366]:
# a_name(分點1)、b_name(分點2)自訂!!!
a_name = '凱基'
b_name = '凱基-台北'

# 建立存檔案的資料夾
path_go = f'{os.getcwd()}\\分點明細查詢\\{a_name}\\{b_name}'   
if not os.path.exists(path_go) : 
    os.makedirs(path_go)
    
# 要跑哪些日期
fnames = glob.glob(path_go + r'\*.xlsx', recursive=False)
if len(fnames)!=0:
    donedate = [fname.split('\\')[-1].split('_')[-1].split('.')[0] for fname in fnames][-1]
    startday = datetime.datetime.strptime(donedate, "%Y-%m-%d")+ datetime.timedelta(days=1)
    dates = chinese_calendar.get_workdays(startday, datetime.datetime.now().date())
else:
    dates = chinese_calendar.get_workdays(datetime.date(2021, 1, 1), datetime.datetime.now().date())
print(f'要跑這些日期: {[str(date) for date in dates]}')
print(f'共 {len(dates)} 個')

要跑這些日期: ['2021-02-07', '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-18', '2021-02-19', '2021-02-20', '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25', '2021-02-26', '2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05', '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11', '2021-03-12', '2021-03-15', '2021-03-16', '2021-03-17', '2021-03-18', '2021-03-19', '2021-03-22', '2021-03-23', '2021-03-24', '2021-03-25', '2021-03-26', '2021-03-29', '2021-03-30', '2021-03-31', '2021-04-01', '2021-04-02', '2021-04-06', '2021-04-07', '2021-04-08', '2021-04-09', '2021-04-12', '2021-04-13', '2021-04-14', '2021-04-15', '2021-04-16', '2021-04-19', '2021-04-20', '2021-04-21', '2021-04-22', '2021-04-23', '2021-04-25', '2021-04-26', '2021-04-27', '2021-04-28', '2021-04-29', '2021-04-30', '2021-05-06', '2021-05-07', '2021-05-08', '2021-05-10', '2021-05-11', '2021-05-12', '2021-05-13', '2021-05-14', '2021-05-17', '2021-05-18', '2021-05-19', '2021-05-20', '2021-05-21', '2021-05-24

## 開始

In [ ]:
a = number1[name1.index(a_name)] #對應的號碼
b = number2[name2.index(b_name)]

for date in dates:
    files = query(a, b, e=date)
    if isinstance(files, type(None)):
        print(f'查無 {str(date)} 資料')
        continue
    else:
        filename = str(b_name) + '_' + str(date)
        writer = pd.ExcelWriter(f'{path_go}\\{filename}.xlsx')
        idx = 0
        for file in files:
            file.to_excel(writer, sheet_name = sheet[idx], index=False)
            idx += 1
        writer.save()
        writer.close()
        print(f'成功匯出 {filename}')
    time.sleep(1)

查無 2021-02-07 資料
查無 2021-02-08 資料
查無 2021-02-09 資料
查無 2021-02-10 資料
成功匯出 凱基-台北_2021-02-18
成功匯出 凱基-台北_2021-02-19
查無 2021-02-20 資料
成功匯出 凱基-台北_2021-02-22
成功匯出 凱基-台北_2021-02-23
成功匯出 凱基-台北_2021-02-24
成功匯出 凱基-台北_2021-02-25
